In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [7]:
def plot_sign():
    x = np.linspace(-8, 8, 100)
    y = np.sign(x)
    plt.ylim(-1.1, 1.1)
    plt.plot(x, y)

    ax = plt.gca()
    ax.set_xlabel(r'${\bf w} \cdot {\bf x}$')
    ax.set_ylabel(r'$y$')
    
    plt.title(r'Perceptron class label as a function of ${\bf w} \cdot {\bf x}$')

    plt.show()

In [5]:
def plot_logistic(generic = True):
    x = np.linspace(-8, 8, 100)
    y = sigmoid(x)
    plt.ylim(-0.1, 1.1)
    plt.plot(x, y)
    plt.axvline(x = 0, ls = '--', lw = 0.5, color = 'black')
    plt.axhline(y = 0.5, ls = '--', lw = 0.5, color = 'black')
    ax = plt.gca()
    if generic:
        ax.set_xlabel(r'$x$')
        ax.set_ylabel(r'$\sigma(x)$')
    else:
        ax.set_xlabel(r'${\bf w} \cdot {\bf x}$')
        ax.set_ylabel(r'$p_{\bf w} ^{1}({\bf x})$')
    plt.show()

In [9]:
def plot_MLE():
    x = np.linspace(0.01, 0.99, 100)
    y = np.power(x, 7) * np.power(1 - x, 3)
    ax = plt.gca()
    ax.set_xlabel(r'$\theta$')
    ax.set_ylabel(r'$L(D; \theta)$')
    plt.title(r'Probability of 7 heads and 3 tails as a function of $p({\tt heads})$')
    plt.plot(x, y)
    plt.show()    

In [12]:
def plot_ln():
    x = np.linspace(0.01, 0.99, 100)
    y = np.log(x)
    ax = plt.gca()
    ax.set_xlabel(r'$p$')
    ax.set_ylabel(r'$\log(p)$')
    plt.title(r'The log is monotone with respect to its argument')
    plt.plot(x, y)
    plt.show()    